In [10]:
from __future__ import annotations
import numpy as np
from qiskit import QuantumCircuit
from qiskit_algorithms import AmplitudeEstimation, EstimationProblem

In [11]:
def create_circuit(num_qubits: int) -> QuantumCircuit:
    ae = AmplitudeEstimation(
        num_eval_qubits=num_qubits - 1,  # -1 because of the to be estimated qubit
    )
    problem = get_estimation_problem()

    qc = ae.construct_circuit(problem)
    qc.name = "ae"
    qc.measure_all()
    return qc

In [12]:
class BernoulliQ(QuantumCircuit):  
# type: ignore[misc]
    """A circuit representing the Bernoulli Q operator."""

    def __init__(self, probability: float) -> None:
        """Initialize the Bernoulli Q operator."""
        super().__init__(1)  # circuit on 1 qubit

        self._theta_p = 2 * np.arcsin(np.sqrt(probability))
        self.ry(2 * self._theta_p, 0)

    def __eq__(self, other: object) -> bool:
        """Return if the operators are equal."""
        return isinstance(other, BernoulliQ) and self._theta_p == other._theta_p

    def __hash__(self) -> int:
        """Return a hash of the operator."""
        return hash(self._theta_p)

    def power(self, power: float, _matrix_power: bool = True) -> QuantumCircuit:
        """Return a circuit implementing the power of the operator."""
        q_k = QuantumCircuit(1)
        q_k.ry(2 * power * self._theta_p, 0)
        return q_k

In [13]:
def get_estimation_problem() -> EstimationProblem:
    """Returns a estimation problem instance for a fixed p value."""
    p = 0.2

    """A circuit representing the Bernoulli A operator."""
    a = QuantumCircuit(1)
    theta_p = 2 * np.arcsin(np.sqrt(p))
    a.ry(theta_p, 0)

    """A circuit representing the Bernoulli Q operator."""
    q = BernoulliQ(p)

    return EstimationProblem(
        state_preparation=a,  # A operator
        grover_operator=q,  # Q operator
        objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
    )